## Helper

### Imports

In [125]:
from manim import *
import numpy as np
from manim_slides import Slide

### Deadish

#### (commented out) Add LaTeX to Path

In [126]:
# import os 
# print(os.environ["PATH"])
# os.environ["PATH"] = os.environ["PATH"].replace(r":/mnt/c/Users/irona/AppData/Local/Programs/MiKTeX/miktex/bin/x64/", "")
# os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2025/bin/x86_64-linux'
# print(os.environ["PATH"])

# If you're having trouble with "latex not found" kind of issues, just close and reopen vscode (idk why it works, but it does)


#### Tex Preamble (only used in qcircuit slide)

In [127]:
# Preamble
preamble_path = "/home/ronakr/Documents/inplaceoracles/Inverting-a-Permutation-with-an-In-Place-Oracle/preamble.tex"
with open(preamble_path, 'r') as file:
    preamble = file.read()
myTemplate = TexTemplate()
myTemplate.add_to_preamble(preamble)
# myTemplate.add_to_preamble(r"\usepackage{ragged2e}")
# 
# myTemplate.add_to_preamble(rf"\geometry{{reset, legalpaper, textwidth={DOCUMENT_TEXT_WIDTH}in}}")

TexTemplate(_body='', tex_compiler='latex', description='', output_format='.dvi', documentclass='\\documentclass[preview]{standalone}', preamble='\\usepackage[english]{babel}\n\\usepackage{amsmath}\n\\usepackage{amssymb}\n\\usepackage[utf8]{inputenc}\n\n\\usepackage[letterpaper,margin=1in]{geometry}\n\\usepackage[OT1]{fontenc}\n\n\\usepackage{amsmath,amssymb,amsthm,amsfonts,latexsym,bbm,xspace,thm-restate}\n\\usepackage{graphicx,float,mathtools,braket,slantsc}\n\\usepackage{enumitem,booktabs,forest,mathdots,soul}\n\\usepackage[useregional]{datetime2}\n\\DTMusemodule{english}{en-US}\n\n\\usepackage{qcircuit,tikz}\n\n\\usepackage[colorlinks,citecolor=blue,,linkcolor=magenta,bookmarks=true]{hyperref}\n\\usepackage[capitalise]{cleveref}\n\n\\usepackage{multicol,array}\n\\usepackage{lipsum,framed}\n\n\\newtheorem{theorem}{Theorem}\n\\newtheorem*{theorem*}{Theorem}\n\\newtheorem{lemma}[theorem]{Lemma}\n\\newtheorem{claim}[theorem]{Claim}\n\\newtheorem{proposition}[theorem]{Proposition}\n\\ne

### Colorcode

In [128]:
RGB = lambda COLOR : ','.join(map(str,COLOR.to_rgb()))

In [129]:
def colorcode(s):
    for keyword in ['in-place', 'In-place', 'In-Place', 'P_f']:
        s = s.replace(keyword, rf" {{\color{{IN_PLACE}} {keyword}}} ")
    for keyword in ['XOR', 'S_f']:
        s = s.replace(keyword, rf" {{\color{{X_O_R}} {keyword}}} ")
    return s

### tex_setup

In [130]:
def tex_setup():
    config.tex_template.add_to_preamble(r"\usepackage{xcolor}")
    config.tex_template.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    config.tex_template.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")

tex_setup()

### Col Tex Template

In [131]:
DOCUMENT_TEXT_WIDTH = 4.85

def col_template(width=1):
    colTemplate = TexTemplate(
        documentclass=r"\documentclass[preview]{standalone}"
    )
    colTemplate.add_to_preamble(r"\usepackage{xcolor}")
    colTemplate.add_to_preamble(r"\usepackage{ragged2e}")
    colTemplate.add_to_preamble(r"\usepackage{geometry}")
    colTemplate.add_to_preamble(rf"\geometry{{textwidth={DOCUMENT_TEXT_WIDTH*width}in}}")
    colTemplate.add_to_preamble(rf"\definecolor{{IN_PLACE}}{{rgb}}{{ {RGB(IN_PLACE)} }}")
    colTemplate.add_to_preamble(rf"\definecolor{{X_O_R}}{{rgb}}{{ {RGB(XOR)} }}")
    return colTemplate

### Definition

In [ ]:
class Definition(VGroup):
    def __init__(
        self,
        header,
        tex,
        box_color=None,
        **kwargs
    ):
        header = Tex(colorcode(header))
        tex = MathTex(colorcode(tex))
        content = VGroup(header, tex).arrange(DOWN)
        super().__init__(
            SurroundingRectangle(content, color=box_color, buff=0.3), 
            content,
            **kwargs
        )

### Bullets

In [132]:
BULLET_SEP = 1
BULLET_TEX_SCALE = 0.7

class Bullets(VGroup):
    
    def __init__(
        self,
        *tex_list, 
        slide=None, 
        width=1, 
        font_scale=1, 
        align_ref=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.colTemplate = col_template(width/font_scale)
        self.tex_scale = BULLET_TEX_SCALE*font_scale

        self.align_ref = align_ref

        self._prior_newlines = BULLET_SEP*1.5

        for tex in tex_list:
            self.add_line(tex)

        if slide is not None:
            self.play(slide)
            

    def add_line(self, tex):
        if isinstance(tex, int):
            self._prior_newlines = BULLET_SEP * (1 + tex)
            return

        tabs = 0
        while tex[0] == '_':
            tabs += 1
            tex = tex[1:]
        bulletMe = False
        if tex[0] == '.':
            bulletMe = True
            tex = tex[1:]

        tex = colorcode(tex)
        tex = rf"\RaggedRight{{ {tex} }}"
        tex_obj = Tex(
            tex, 
            tex_template=self.colTemplate,
        ).scale(self.tex_scale)

        bullet = MathTex("\\bullet").scale(self.tex_scale)
        if bulletMe:
            bullet.next_to(tex_obj, LEFT).align_to(tex_obj, UP)
            bullet.shift(DOWN*0.5*bullet.height)
            tex_obj = VGroup(bullet, tex_obj)

        if self.align_ref is not None:
            prev = self[-1] if len(self) > 0 else self.align_ref
            tex_obj.next_to(prev, self._prior_newlines * DOWN)
            tex_obj.align_to(self.align_ref, LEFT)
        else:
            self.align_ref = tex_obj
        
        tex_obj.shift(RIGHT*tabs*(DEFAULT_MOBJECT_TO_MOBJECT_BUFFER + bullet.width))
            
        self._prior_newlines = BULLET_SEP

        self.add(tex_obj)

    def play(self, slide, animation=FadeIn):
        for line in self:
            slide.play(animation(line))
            slide.next_slide()

### Steps

In [ ]:
class Steps(VGroup):

    def __init__(
        self,
        scale_factor=BULLET_TEX_SCALE,
        **kwargs,
    ) -> None:
        super().__init__(**kwargs)
        self.scale_factor = scale_factor
        

    def index(arr, key):
        if isinstance(key, int):
            return arr[key]
        for i in key:
            arr = arr[i]
        return arr
    
    
    def prev_cause(self):
        return None if len(self[-1]) == 1 else self[-1][0]
    
    
    def prev_result(self):
        return self[-1][-1]
    

    def add_step(
        self,
        *results,
        cause=None,
        **kwargs,
    ):
        result_mobj = MathTex(*results, **kwargs).scale(self.scale_factor)
        if len(self) > 0:
            result_mobj.next_to(self.prev_result(), DOWN).align_to(self.prev_result(), LEFT)
            if cause is None:
                cause = f"\\xrightarrow{{\\ \\ \\ }}"
            elif cause != '=':
                cause = f"\\xrightarrow{{ {cause} }}"

        self.add(VGroup())
        if cause is not None:
            cause_mobj = MathTex(cause).scale(self.scale_factor).next_to(result_mobj, LEFT)
            cause_mobj.shift(UP*cause_mobj.height*0.4)
            self[-1].add(cause_mobj)
        self[-1].add(result_mobj)
        
        return self

    def play(self, slide, indices=[-1], animation=FadeIn):
        slide.play(*[animation(self.index(i)) for i in indices])
        slide.next_slide()

### ShowTexIndices

In [134]:
def ShowTexIndices(self, tex):
    # Observe first level labels
    tex_ = tex.copy().next_to(tex, DOWN)
    self.add(index_labels(tex_))

    # Observe second level labels
    tex__ = tex_.copy().next_to(tex_, DOWN)
    for part in tex__:
        self.add(index_labels(part))

### FlipTransform

In [135]:
def FlipTransform(self, mobject_from=None, mobject_to=None):
    self.play(Homotopy(lambda x,y,z,t : (np.cos(t*np.pi/2)*x, y + 0.1*x*np.sin(t*np.pi/2), z), mobject_from, rate_func=rush_into, run_time=0.8))
    self.remove(mobject_from)
    self.play(Homotopy(lambda x,y,z,t : (np.sin(t*np.pi/2)*x, y - 0.1*x*np.cos(t*np.pi/2), z), mobject_to, rate_func=rush_from, run_time=0.8))

In [136]:
# def _flip(x,y,z,t):
#     return (np.cos(t*np.pi)*x, 0.5*x*np.sin(t*np.pi)+y, z)

# def Flip(mobject=None, *vargs, **kwargs):
#     return Homotopy(_flip, mobject, *vargs, **kwargs)

### GroverQuery

In [137]:
def GroverQuery(self, bar_chart):
    def _negate(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*y*np.sin(t*np.pi), np.cos(t*np.pi)*y)
    idx = 0 # index of bar of marked element
    bar = bar_chart.bars[idx]
    self.play(Homotopy(_negate, bar, rate_func=rush_into, run_time=0.8))
    bar_chart.bars.remove(bar)
    bar_chart.bars.insert(idx, bar_chart._create_bar(idx, -bar_chart.values[idx]))
    bar_chart._update_colors()
    bar_chart.values[0] *= -1

### GroverDiffuseTransform

In [138]:
def GroverDiffuseTransform(self, bar_chart, new_values):
    avg = np.average(bar_chart.values)
    def _diffuse_from(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x - 0.3*(y-avg)*np.sin(t*np.pi/2), np.cos(t*np.pi/2)*(y-avg) + avg)
    def _diffuse_to(x,y,z,t):
        x,y = bar_chart.point_to_coords([x,y,z])
        return bar_chart.coords_to_point(x + 0.3*(y-avg)*np.cos(t*np.pi/2), np.sin(t*np.pi/2)*(y-avg) + avg)
    
    self.play(*[Homotopy(_diffuse_from, bar, rate_func=rush_into, run_time=0.8) for bar in bar_chart.bars])
    for i,bar in enumerate(bar_chart.bars):
        bar_chart.bars.remove(bar)
        bar_chart.bars.insert(i, bar_chart._create_bar(i, new_values[i]))
    bar_chart._update_colors()
    bar_chart.values[:len(new_values)] = new_values
    self.play(*[Homotopy(_diffuse_to, bar, rate_func=rush_from, run_time=0.8) for bar in bar_chart.bars])

### Circuit Braces

In [139]:
def CircuitBraces(ckt, left, right):
    bL = Brace(ckt, LEFT)
    bLt = bL.get_tex(left + r" \rightarrow")
    bR = Brace(ckt, RIGHT)
    bRt = bR.get_tex(r"\rightarrow " + right)
    return VGroup(bL, bLt, bR, bRt)

### OracleCircuit

In [140]:
def OracleCircuit(oracle_tex, left, right, length_tex=r"o(\sqrt{N})", flipped=False):
    flipped = -1 if flipped else 1
    oracle_color, ckt_color = ((IN_PLACE, XOR) if oracle_tex[0] == 'P' else (XOR, IN_PLACE))

    oracle_mathtex = MathTex(oracle_tex)
    oracle_box = SurroundingRectangle(oracle_mathtex, buff=0.2, color=WHITE, fill_opacity=1, fill_color=oracle_color)
    oracle_drawing = VGroup(oracle_box, oracle_mathtex)

    ckt_outline = RoundedRectangle(corner_radius=0.25, height=4.0, width=6.5, fill_opacity=1, fill_color=ckt_color)

    ckt_input = MathTex(left + r" \rightarrow").next_to(ckt_outline, LEFT)
    ckt_output = MathTex(r"\rightarrow " + right).next_to(ckt_outline, RIGHT)
    ckt_input_and_output = VGroup(ckt_input, ckt_output)

    len_line = NumberLine(length=ckt_outline.width, x_range=[0, 1, 1]).next_to(ckt_outline, UP)
    len_label = MathTex(length_tex).next_to(len_line, UP)
    ckt_length = VGroup(len_line, len_label)
    # DoubleArrow(pt1, pt2, tip_shape_end=LineArrowTip, tip_shape_start=LineArrowTip, tip_length=0.1).next_to(ckt_outline, UP)
    
    oracles = VGroup()
    oracles.add(oracle_drawing.move_to(ckt_outline.get_center()).shift(flipped*2.3*LEFT + UP))
    oracles.add(oracles[0].copy().shift(flipped*1.5*RIGHT + DOWN))
    oracles.add(oracles[0].copy().shift(flipped*3*RIGHT))
    oracles.add(oracles[0].copy().shift(flipped*4.5*RIGHT + 2*DOWN))

    ckt = VGroup(ckt_outline, ckt_input_and_output, ckt_length, oracles)

    return ckt

## Tests

### Transform Sanity Check

In [99]:
%%manim -ql SanityCheck

class SanityCheck(Scene):
    def construct(self):
        s0 = MathTex(r"a")
        s1 = MathTex(r"\frac{a}{b}")
        s2 = MathTex(r"\frac{b}{a}")
        tms0 = TransformMatchingShapes(s0,s1, key_map={-6351953992051140758:8757917165807693890})
        tms = TransformMatchingShapes(s1,s2)#, key_map={-6351953992051140758:8757917165807693890})
        # s1a = tms.get_mobject_key(tms.get_mobject_parts(s1)[0])
        # s2l = tms.get_mobject_key(tms.get_mobject_parts(s2)[1])
        # tms.key_map = {s1a:s2l}
        print(tms0.get_mobject_parts(s0))
        print(tms0.get_mobject_parts(s1))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s0)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[0]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[1]))
        print(tms0.get_mobject_key(tms0.get_mobject_parts(s1)[2]))

        print(tms.get_mobject_parts(s1))
        print(tms.get_mobject_parts(s2))

        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s1)[2]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[0]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[1]))
        print(tms.get_mobject_key(tms.get_mobject_parts(s2)[2]))
        self.play(tms0)
        self.play(tms)

Manim Community v0.19.0

[09/06/25 21:01:12] INFO     Writing a to media/Tex/065e874364a44b8a.tex                    ]8;id=511844;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=557357;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\

KeyboardInterrupt: 

### Text Test

In [ ]:
%%manim -ql TextTest

class TextTest(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        title = Title(colorcode("Search with In-Place Queries"), tex_template=col_template())
        self.add(title)
        self.next_slide()

        # paragraph = Bullets(
        #     r"This is a long string of in-place text meant to span $P_f$ the entire width ${\color{IN_PLACE} x^2}$ of the XOR slide from one margin to the other, at least, that's the hope. Please note if that is not what is observed!", 
        #     align_ref=title
        # )

        # paragraph.play(self)

        bullets = Bullets(
            # "Hello, my name is bob",
            # "Hello, my name is bob",
            # 1,
            # "This sentence is meant to test that text wrapping works properly.",
            ". this is a bullet",
            "_this is an indented bullet \\\\ across multiple lines!! \\\\ So many lines...",
            "_.this is a nested bullet",
            "__.and another",
            "_.and another",
            "$\\phi + 2000$",
            slide=self, width=0.3, font_scale=0.5, align_ref=title
        )

Manim Community v0.19.0

[09/06/25 15:38:38] INFO     Animation 0 : Using cached data (hash :                           ]8;id=215286;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=651370;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_1246782520_2947850686)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=571992;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=600427;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_939913858)                                                      

[09/06/25 15:38:41] INFO     Animation 2 : Partial movie file written in                   ]8;id=850787;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=986984;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/1905288516_3302847627_285                         
                             675698.mp4'                                                                           

[09/06/25 15:38:43] INFO     Animation 3 : Partial movie file written in                   ]8;id=165887;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=933451;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TextTest/1905288516_1443166991_149                         
                             1729649.mp4'                                                                          

...

                    INFO     Rendered TextTest                                                         ]8;id=152807;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=568726;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 12 animations                                                                  

[09/06/25 15:39:10] INFO     Generated 6 slides to                                                      ]8;id=640504;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=444249;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

[09/06/25 15:39:10] INFO     Generated 6 slides to                                                      ]8;id=340093;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=12354;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TextTest'                            

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=180013;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=247364;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

                    INFO     Slide 'TextTest' configuration written in                                  ]8;id=714844;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=649284;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TextTest.json'                             

## Overview

### Query Complexity (motivation)
- What is a problem? 
  - Input: some f, Output: some property of f. 
  - Egs: 
    - (1) Given: N-bit integer. Output: is it nonzero? (show arrow to "integer", saying f : [N] -> {0,1}, transform) 
      (show after (2)) Given: function f : [N] -> {0,1}. Output: is there an x such that f(x) = 1?
      mention this is "searching for a 1"
    - (2) Given: a shuffled deck of cards labeled {1,...,N}. Output: location of the "1" card.
      Given permutation f: [N] -> [N]. Output: x such that f(x) = 1.
- "In Complexity theory we often care about how much time the best algorithm to solve a problem would take as the size of the input (N) increases"
- "That's really hard to lower bound. The best technique we have is query complexity."
- In query cxty, we imagine the input is behind a paywall, and we need to pay an oracle every time we want to access bits of it. (oracle pic)
  - I will tell the oracle x and the oracle will tell me f(x), each request is called a query
  <!-- the label of the xth card in the deck. (exchange) -->
  - It's fairly intuitive that, for these specific problems above, N queries are necessary classically, (implying an Omega(N) lower bound on time?).
  - How do we do quantum query complexity? 
    - Our first thought might be |x> -> |f(x)>. 
      - Violates QM if f is not reversible. 
    - Solution: |x>|a> -> |x>|a + f(x)>. -self inverse -equiv to phase oracles
    - Okay, but what if f is a permutation? 
    - Then |x> -> |f(x)> is fine, called in-place queries.

### Key Question of this talk
- Given a permutation f, how do XOR queries and in-place queries to f compare?
  
  <!-- however, the ability to make multiple queries in superposition lets us find "1" with O(sqrt(N)) queries, through what's called Grover's algorithm
  - Understanding this alg is crucial to understanding our alg, so let's briefly review -->

### Table of comparisons
- If we have access to both f and f^{-1}, then both models can simulate the other
- Omega(sqrt(N)) XOR queries required to exactly simulate an in-place query
  - In this paper: Omega(sqrt(N)) in-place to XOR
- IndexErasure (state conversion) O(sqrt(N)) vs O(1)
  - states of the form |x>|f(x)> -> |0>|f(x)>
- SetComp (promise decision problem) O(N^{1/7}) vs O(1)
  - venn diagram highlighting: two large sets are the same or have large symmetric difference
- Unstructured search (search problem) O(sqrt(N)) vs conjectured Omega(N). (cross out conj, write ours) In this paper: O(sqrt(N)).
  - N shuffled cards, locate "1"
- Trashy Simon's (promise search problem) O(log(N)) vs conjectured Omega(sqrt(N)) ### TODO: verify sqrt(N) correct!

### Grover's
- Our problem: Given a shuffled deck of cards labeled [N], find 1. 
- (Show 12345 example truth table with f, circle inv of 1) 
- (transform problem) given a permutation f : [N] -> [N], find x^* := f^{-1}(1).
- (disappear truth table)
- In Grover's algorithm, we start off with a uniform superposition over all x in [N] (bar chart)
- Repeatedly alternate between the following two operations:
  - Query, which negates the amplitude on x^* (flip)
  - Grover's Diffusion operator, which flips all amplitudes about their average (draw line, flip)
  - (quickly evolve a few times until x^* peaks) after ~sqrt N iterations of this, we will have made ~sqrt N queries and the probability of seeing x^* when we measure.

### Where Grover's fails
- key step is being able to take |x> -> (-1)^{f(x) = 1} |x>
- |x>|0> -> |x>|f(x)> -> (-1)^{f(x) = 1}|x>|f(x)> (->) |x>|0>  <- (new slide) function erasure is hard
- Without being able to erase f, interference fails

### In fact, we show func erasure is hard!
- pf omitted here.

### Our Alg
- Do the math, have a box reminding people what in-place and XOR do
- |x> -> |x>|x>|0> -> |x>|f(x)>|0> -> |x*>|1>|1> + sum|x>|f(x)>|0>
- -> |x*>|1>|1> + sum|f(x)>|f(x)>|0> -> |x*>|1>|1> + sum|f(x)>|0>|0>  -> |x*>|1>|1> + sum|f(x)>|1>|0> -> |x*>|1> + sum|f(x)>|0>
- convert to bars

### Step Thru Our Alg
- show bars

### Kashefi
- 
- Key fact: perm inv LBs for both Sf [Ambainis] and Pf [FK18]

### Kashefi Pfs (3 slides)
- Inverting circuits
- Kashefi arg (|x>|a> -[Sf]- |x>|f(x)> in corner so we can flip it to drive home that Sf inv = Sf)
- Our arg (|x> -[Pf]- |f(x)> in corner so we can flip it to drive home that Pf inv = P{f_inv})
- => function erasure is hard (analog of index erasure)

### Sf Pf box
- Grovers + Kashefi => Theta(sqrt(N)) XOR to in-place
- Our Alg + our arg => Theta(sqrt(N)) in-place to XOR

### Summary
- Unstructured Search takes Theta(sqrt(N)) queries (in-place or XOR) (algorithm)
- Function Erasure takes O(1) XOR Queries and O(sqrt(N)) in-place queries (kashefi)
- More generally, Sf Pf Sfinv Pfinv all req sqrt(N) queries to simulate each other
- Future work: decision problem separations
  - candidates: simon, emb perm inv


## TitleSlide

In [ ]:
%%manim -ql TitleSlide

class TitleSlide(Slide):
    def construct(self):
        # self.wait_time_between_slides = 1
        title = Tex(r"\textsc{Quantum Search with In-Place Queries}").scale(1.2).shift(2*UP)
        line = Underline(title)
        subtitle = Tex(r"Blake Holman, \textbf{Ronak Ramachandran}, and Justin Yirka").scale(0.75)
        subtitle.next_to(title, DOWN, buff=0.5)
        ronak = ImageMobject("ronak.png").scale_to_fit_height(2.5).next_to(subtitle, DOWN, buff=0.5)
        blake = ImageMobject("blake.jpg").scale_to_fit_height(2.5).next_to(ronak, LEFT, buff=1)
        justin = ImageMobject("jirka.jpg").scale_to_fit_height(2.5).next_to(ronak, RIGHT, buff=1)
        remail = Tex(r"ronakr@utexas.edu").next_to(ronak, DOWN).scale(0.5)
        bemail = Tex(r"holman14@purdue.edu").next_to(blake, DOWN).scale(0.5)
        jemail = Tex(r"justin@blanqet.net").next_to(justin, DOWN).scale(0.5)
        # self.play(Add(title), Write(subtitle), Succession(FadeIn(blake), FadeIn(ronak), FadeIn(justin), run_time=1.8))
        self.add(
            title, line, subtitle,
            blake, ronak, justin,
            bemail, remail, jemail
        )
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/06/25 14:39:04] INFO     Animation 0 : Partial movie file written in                   ]8;id=798335;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=378612;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/TitleSlide/931832650_722531385_109                         
                             9345004.mp4'                                                                          

                    INFO     Combining to Movie file.                                      ]8;id=780842;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=494782;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=941858;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=818593;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/TitleSlide.mp4'                                                                
                                                                                                                   

                    INFO     Rendered TitleSlide                                                       ]8;id=313510;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=621081;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 1 animations                                                                   

[09/06/25 14:39:04] INFO     Generated 1 slides to                                                      ]8;id=765006;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=21560;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

                    INFO     Generated 1 slides to                                                      ]8;id=238592;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=835976;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/TitleSlide'                          

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=397973;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=928974;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

                    INFO     Slide 'TitleSlide' configuration written in                                ]8;id=566775;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=601232;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/TitleSlide.json'                           

# Introduction

## Query Models

In [ ]:
%%manim -ql QueryModels

class QueryModels(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.next_slide()

## Comparison Table

In [158]:
%%manim -ql ComparisonTable

class ComparisonTable(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Are in-place queries stronger than XOR queries?"))
        self.add(title)
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        def ul(upper, ucite, lower, lcite):
            u = Tex(rf"$O({upper})$ [{ucite}]")
            l = Tex(rf"$\Omega({lower})$ [{lcite}]").next_to(u, DOWN).align_to(u, LEFT)
            return VGroup(u, l)

        xp_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "Amb02")
        xi_ul = ul(r"\sqrt{N}", "Gro96", r"\sqrt{N}", "AMRR11")
        xf_ul = MathTex(rf"\Theta(1)").scale(2)
        ip_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", "FK18][BY23")
        ii_ul = MathTex(rf"\Theta(1)").scale(2)
        if_ul = ul(r"\sqrt{N}", r"H\textbf{R}Y25", r"\sqrt{N}", r"H\textbf{R}Y25")

        ip_ul[0].set_color(RED)
        if_ul.match_color(ip_ul[0])
        func_erase_defn.match_color(ip_ul[0])

        table = VGroup(
            VMobject(), idx_erase_defn, perm_inv_defn, func_erase_defn,
            xor_defn, xi_ul, xp_ul, xf_ul,
            in_place_defn, ii_ul, ip_ul, if_ul,
        )

        table.arrange_in_grid(
            col_alignments="rccc",
            row_alignments="dcc",
        )

        conj = Tex(
            rf"[FK18]: Is $O(N)$ optimal?"
        ).next_to(ip_ul[1], UP).align_to(ip_ul[1], LEFT)

        (table+conj).match_width(title).shift(UP*0.5)
        cross = Cross(conj)

        open_prob = Tex(
            colorcode(r"STILL OPEN: \\ Does any \textbf{decision problem} require more in-place queries than XOR queries?")
        ).scale(BULLET_TEX_SCALE).next_to(table, DOWN*2)
        
        def fadein(*x):
            self.play(*[FadeIn(item) for item in x])
            self.next_slide()

        fadein(xor_defn, in_place_defn)
        fadein(idx_erase_defn)
        fadein(ii_ul)
        fadein(xi_ul[1])
        fadein(xi_ul[0])

        fadein(perm_inv_defn)
        fadein(xp_ul)
        fadein(ip_ul[1])
        fadein(conj)
        fadein(cross)
        self.play(FadeOut(conj), FadeOut(cross), FadeIn(ip_ul[0]))
        self.next_slide()

        fadein(func_erase_defn)
        fadein(xf_ul)
        fadein(if_ul[0])
        fadein(if_ul[1])

        fadein(open_prob)

Manim Community v0.19.0

[09/07/25 01:41:59] INFO     Animation 0 : Partial movie file written in                   ]8;id=911611;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=480033;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/931832650_23700055                         
                             1_3843083297.mp4'                                                                     

[09/07/25 01:42:01] INFO     Animation 1 : Partial movie file written in                   ]8;id=42130;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=445861;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_1443166                         
                             991_2510673923.mp4'                                                                   

[09/07/25 01:42:04] INFO     Animation 2 : Partial movie file written in                   ]8;id=711647;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=543155;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_6958477                         
                             15_3059450685.mp4'                                                                    

[09/07/25 01:42:07] INFO     Animation 3 : Partial movie file written in                   ]8;id=336373;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=433719;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/ComparisonTable/1905288516_1443166                         
                             991_3160561489.mp4'                                                                   

...

                    INFO     Rendered ComparisonTable                                                  ]8;id=651313;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=203082;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 32 animations                                                                  

[09/07/25 01:47:26] INFO     Generated 16 slides to                                                     ]8;id=425274;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=304053;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ComparisonTable'                     

[09/07/25 01:47:26] INFO     Generated 16 slides to                                                     ]8;id=44142;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=449423;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/ComparisonTable'                     

                    INFO     Slide 'ComparisonTable' configuration written in                           ]8;id=241775;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=349022;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ComparisonTable.json'                      

                    INFO     Slide 'ComparisonTable' configuration written in                           ]8;id=491523;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=679125;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/ComparisonTable.json'                      

## Results

In [142]:
%%manim -ql Results

class Results(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        ### Known Results
        title = Title("Known Results")
        self.add(title)
        self.next_slide()

        xor_defn = Definition(
            r"XOR Query",
            r"|x\rangle |a\rangle \xrightarrow{S_f} |x\rangle |a \oplus f(x)\rangle",
            box_color=XOR,
        )

        in_place_defn = Definition(
            r"In-Place Query",
            r"|x\rangle \xrightarrow{P_f} |f(x)\rangle",
            box_color=IN_PLACE,
        )

        idx_erase_defn = Definition(
            r"\textsc{IndexErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |0\rangle |f(x)\rangle",
        )

        func_erase_defn = Definition(
            r"\textsc{FunctionErasure}",
            r"|x\rangle |f(x)\rangle \rightarrow |x\rangle |0\rangle",
        )

        perm_inv_defn = Definition(
            r"\textsc{PermutationInversion}",
            r"\text{Given $f$, find $f^{-1}(1)$}",
        )

        idx_erase_defn[0].match_height(perm_inv_defn[0])
        func_erase_defn[0].match_height(perm_inv_defn[0])

        sep = 1.5
        oracle_defns = VGroup(
            xor_defn, in_place_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep).next_to(title, DOWN*sep)
        prob_defns = VGroup(
            func_erase_defn, idx_erase_defn, perm_inv_defn
        ).scale(BULLET_TEX_SCALE).arrange(RIGHT*sep)
        (prob_defns-func_erase_defn).next_to(oracle_defns, DOWN*sep)
        func_erase_defn.next_to(idx_erase_defn, LEFT*sep)

        self.play(FadeIn(oracle_defns), FadeIn(prob_defns - func_erase_defn)) #TODO animate sequentially

        known_bullets = Bullets(
            r".\textsc{PermutationInversion} requires $\Omega(\sqrt{N})$ queries {\color{X_O_R} [Amb02]}{\color{IN_PLACE} [FK18]}",
            r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ XOR queries {\color{X_O_R} [Amb02]}",
            r"_.{\color{IN_PLACE} [FK18]} Conjecture $\Omega(N)$ in-place queries"
            r".Simulating an in-place query requires $\Omega(\sqrt{N})$ XOR queries",
            r".\textsc{IndexErasure} requires $\Omega(\sqrt{N})$ XOR queries {\color{X_O_R} [AMRR11]}",
            # r".A controlled in-place query can be implemented with one in-place query",
        ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)


        ### Our Results
        our_results_title = Title("Our Results")
        self.play(FadeTransform(title, our_results_title))
        self.next_slide()

        # bullets = Bullets(
        #     r".\textsc{PermutationInversion} possible with $O(\sqrt{N})$ in-place queries",
        #     r".Simulating an XOR query requires $\Omega(\sqrt{N})$ in-place queries",
        #     r"_$\implies$ \textsc{FunctionErasure} requires $\Omega(\sqrt{N})$ in-place queries",
        #     # r".A controlled in-place query can be implemented with one in-place query",
        # ).next_to(prob_defns, DOWN*sep).align_to(title, LEFT)
        
        # self.play(FadeIn(bullets[0]))
        # self.next_slide()
        
        # alg_tight = Tex(r"$\leftarrow$ tight [FK18][BY23]").scale(BULLET_TEX_SCALE*0.8).next_to(bullets[0]).shift(LEFT*0.1)
        # self.play(FadeIn(alg_tight))
        # self.next_slide()

        # self.play(FadeIn(bullets[1]))
        # self.next_slide()

        self.play(FadeIn(func_erase_defn), prob_defns.animate.next_to(oracle_defns, DOWN*sep))
        # self.play(FadeIn(bullets[2]))
        # self.next_slide()

        # LBs_tight = VGroup(
        #     Brace(bullets[1:], RIGHT), 
        #     Tex(r"tight by above \\ algorithm").scale(BULLET_TEX_SCALE*0.8)
        # ).arrange(RIGHT).next_to(bullets[1]).align_to(bullets[1], UP)
        # self.play(FadeIn(LBs_tight))
        # self.next_slide()

Manim Community v0.19.0

[09/06/25 23:02:13] INFO     Writing Known Results to media/Tex/614c8a19c9a51b50.tex        ]8;id=364640;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=914504;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\

[09/06/25 23:02:15] INFO     Writing {\color{X_O_R} XOR}  Query to                          ]8;id=908358;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=567;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\
                             media/Tex/c43d616b067e5960.tex                                                        

[09/06/25 23:02:16] INFO     Writing |x\rangle |a\rangle \xrightarrow{ {\color{X_O_R} S_f}  ]8;id=767165;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=961363;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\
                             } |x\rangle |a \oplus f(x)\rangle to                                                  
                             media/Tex/28f4758bde3315e1.tex                                                        

[09/06/25 23:02:18] INFO     Writing {\color{IN_PLACE} In-Place}  Query to                  ]8;id=781858;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=928491;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/utils/tex_file_writing.py#111\111]8;;\
                             media/Tex/d5e73a1425e29963.tex                                                        

...

                    INFO     Rendered Results                                                          ]8;id=486617;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=620356;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 4 animations                                                                   

[09/06/25 23:02:55] INFO     Generated 2 slides to                                                      ]8;id=588670;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=467013;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Results'                             

[09/06/25 23:02:55] INFO     Generated 2 slides to                                                      ]8;id=498924;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=260904;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Results'                             

                    INFO     Slide 'Results' configuration written in                                   ]8;id=971921;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=760633;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Results.json'                              

                    INFO     Slide 'Results' configuration written in                                   ]8;id=11882;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=563244;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Results.json'                              

## Key Question

In [ ]:
%%manim -ql KeyQuestion

class KeyQuestion(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        question = Tex(
            r"Given a \textbf{permutation} $f$, how do \textbf{XOR queries} \\ and \textbf{in-place queries} to $f$ compare?",
            substrings_to_isolate=[r"\textbf{XOR queries}", r"\textbf{in-place queries}"]
        )
        question.set_color_by_tex(r"XOR", XOR)
        question.set_color_by_tex(r"in-place", IN_PLACE)
        self.play(Write(question))
        self.next_slide()

Manim Community v0.19.0

[09/05/25 12:44:02] INFO     Animation 0 : Partial movie file written in                   ]8;id=896656;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=78857;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/KeyQuestion/419893507_3365951919_2                         
                             23132457.mp4'                                                                         

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=864814;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=374730;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/KeyQuestion/3267242197_1443166991_                         
                             1163309912.mp4'                                                                       

                    INFO     Combining to Movie file.                                      ]8;id=509050;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=659398;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=418351;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=916775;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/KeyQuestion.mp4'                                                               
                                                                                                                   

                    INFO     Rendered KeyQuestion                                                      ]8;id=842570;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=86561;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   

[09/05/25 12:44:03] INFO     Generated 1 slides to                                                      ]8;id=721515;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=995935;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

[09/05/25 12:44:03] INFO     Generated 1 slides to                                                      ]8;id=877403;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=384857;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/KeyQuestion'                         

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=950558;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=700215;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

                    INFO     Slide 'KeyQuestion' configuration written in                               ]8;id=328972;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=876114;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/KeyQuestion.json'                          

# Why Grover Fails --> FE LB

## StepThruGroverBar

In [ ]:
def get_Grover_amps(t, N=10):
    a_t = np.sin((2*t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((2*t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [ ]:
%%manim -ql StepThruGroverBar

class StepThruGroverBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/4 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        title = Tex("Grover's Algorithm").next_to(chart, UP)
        self.play(Write(title, run_time=0.5), DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        a_t,b_t = get_Grover_amps(0,N)
        for t in range(1,T):
            # Query
            GroverQuery(self, chart)
            self.next_slide()

            # Average
            avg_line = chart.plot(lambda x : np.average(chart.values), stroke_width=1)
            self.play(Create(avg_line))
            self.next_slide()

            # Diffuse
            a_t,b_t = get_Grover_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            GroverDiffuseTransform(self, chart, values)
            self.play(FadeOut(avg_line))
            self.next_slide()


Manim Community v0.19.0

[09/05/25 12:44:06] INFO     Animation 0 : Partial movie file written in                   ]8;id=732559;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=840269;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/419893507_124811                         
                             1026_223132457.mp4'                                                                   

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=29599;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=113472;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/3267242197_14431                         
                             66991_265397911.mp4'                                                                  

[09/05/25 12:44:07] INFO     Animation 2 : Partial movie file written in                   ]8;id=948324;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=572908;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/3267242197_36854                         
                             93934_824231333.mp4'                                                                  

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=128145;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=186904;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruGroverBar/3267242197_14431                         
                             66991_4242583689.mp4'                                                                 

...

                    INFO     Rendered StepThruGroverBar                                                ]8;id=117553;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=558487;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/05/25 12:44:26] INFO     Generated 7 slides to                                                      ]8;id=65700;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=621452;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

[09/05/25 12:44:26] INFO     Generated 7 slides to                                                      ]8;id=904244;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=117152;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruGroverBar'                   

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=500255;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=723472;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

                    INFO     Slide 'StepThruGroverBar' configuration written in                         ]8;id=597786;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=49941;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruGroverBar.json'                    

## Where Grover Fails

In [ ]:
%%manim -ql GroverBreakdown

class GroverBreakdown(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        tex_setup()

        title = Title(colorcode(r"Why can't we Grover's search with in-place queries?"))
        self.add(title)
        self.next_slide()

        query_tex_cases = MathTex(r"|x\rangle \rightarrow \begin{cases} -|x\rangle & \text{if $f(x) = 1$} \\ |x\rangle & \text{otherwise} \end{cases}")

        query_tex = MathTex(r"|x\rangle \rightarrow (-1)^{f(x)=1}|x\rangle")
        self.play(Write(query_tex_cases))
        self.next_slide()
        self.play(Transform(query_tex_cases, query_tex))
        self.next_slide()

        self.play(query_tex.animate.shift(2*UP))
        steps = []
        steps.append(MathTex(
            r"|x\rangle",
            r"|0\rangle",
        ).shift(UP))
        steps.append(MathTex(
            r"\xrightarrow{S_f}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.append(MathTex(
            r"\xrightarrow{\phantom{S_f}} (-1)^{f(x)=1}",
            r"|x\rangle|f(x)\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps.append(MathTex(
            r"\xrightarrow{S_f} (-1)^{f(x)=1}",
            r"|x\rangle",
            r"|0\rangle",
        ).next_to(steps[-1], DOWN).align_to(query_tex, LEFT))
        steps[0].align_to(steps[1][1], LEFT)
        steps[1][0][0:2].set_fill(color=XOR)
        steps[3][0][0:2].set_fill(color=XOR)

        # ShowTexIndices(self, steps)

        self.play(FadeIn(steps[0][0]))
        self.next_slide()
        self.play(FadeIn(steps[0][1]))
        self.next_slide()
        self.play(FadeIn(steps[1]))
        self.next_slide()
        self.play(FadeIn(steps[2]))
        self.next_slide()
        self.play(FadeIn(steps[3][0]), FadeIn(steps[3][1]), FadeIn(steps[3][2]))
        self.next_slide()
        self.play(FadeOut(steps[3][-1]))
        self.next_slide()

        self.play(Circumscribe(VGroup(steps[2], steps[3]), color=RED_E, fade_out=True))

        func_eras = MathTex(
            r"|x\rangle|f(x)\rangle",
            r"\rightarrow",
            r"|x\rangle",
            r"|0\rangle",
        )
        to_fade = [query_tex, steps[0], steps[1], steps[2][0]]
        self.play(
            FadeOut(*to_fade),
            ReplacementTransform(steps[2][1], func_eras[0]),
            ReplacementTransform(steps[3][0], func_eras[2]),
            ReplacementTransform(steps[3][1], func_eras[3]),
            FadeIn(func_eras[1])
        )
        self.next_slide()

        func_eras_title = Tex("Function Erasure").next_to(func_eras, 2*UP)
        self.play(Write(func_eras_title))
        self.next_slide()


Manim Community v0.19.0

[09/05/25 12:44:29] INFO     Animation 0 : Partial movie file written in                   ]8;id=261447;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=231696;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/419893507_38404887                         
                             23_223132457.mp4'                                                                     

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=516534;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=285968;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/3267242197_1443166                         
                             991_3721481534.mp4'                                                                   

[09/05/25 12:44:30] INFO     Animation 2 : Partial movie file written in                   ]8;id=813178;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=317695;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/3267242197_2775674                         
                             738_2002292858.mp4'                                                                   

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=777068;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=79632;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/GroverBreakdown/3267242197_1443166                         
                             991_844290619.mp4'                                                                    

...

                    INFO     Rendered GroverBreakdown                                                  ]8;id=214518;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=699610;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 22 animations                                                                  

[09/05/25 12:44:47] INFO     Generated 10 slides to                                                     ]8;id=180125;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=823276;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

[09/05/25 12:44:47] INFO     Generated 10 slides to                                                     ]8;id=542617;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=260819;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/GroverBreakdown'                     

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=38642;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=109908;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

                    INFO     Slide 'GroverBreakdown' configuration written in                           ]8;id=249333;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=823577;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/GroverBreakdown.json'                      

## Inverting Circuits

In [ ]:
%%manim -ql InvertingCircuits1

class InvertingCircuits1(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{A} & \multigate{1}{C} & \gate{D} & \qw \\
            & \gate{B} & \ghost{C} & \gate{E} & \qw
        }'''
        abcde_ckt = MathTex(abcde_tex, tex_template=myTemplate)
        abcde_braces = CircuitBraces(abcde_ckt, r"|\psi\rangle", r"\mathcal{C}|\psi\rangle")
        abcde = VGroup(abcde_ckt, abcde_braces)

        self.play(Write(abcde))
        self.next_slide()

        inv_abcde_tex = r'''\Qcircuit @C=0.7em @R=0.7em @!R {
            & \gate{D^{-1}} & \multigate{1}{C^{-1}} & \gate{A^{-1}} & \qw \\
            & \gate{E^{-1}} & \ghost{C^{-1}} & \gate{B^{-1}} & \qw
        }'''
        inv_abcde_ckt = MathTex(inv_abcde_tex, tex_template=myTemplate)
        inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"\mathcal{C} |\psi\rangle", r"|\psi\rangle")
        inv_abcde = VGroup(inv_abcde_ckt, inv_abcde_braces)

        FlipTransform(self, abcde, inv_abcde)
        self.next_slide()

        new_inv_abcde_braces = CircuitBraces(inv_abcde_ckt, r"|\psi\rangle", r"\mathcal{C}^{-1} |\psi\rangle")
        self.play(Transform(inv_abcde_braces, new_inv_abcde_braces))
        # self.play(*[TransformMatchingShapes(inv_abcde_braces[i], new_inv_abcde_braces[i]) for i in (1,3)])
        self.next_slide()
        

Manim Community v0.19.0

[09/05/25 12:44:49] INFO     Animation 0 : Partial movie file written in                   ]8;id=512397;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=692221;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/419893507_27722                         
                             01704_223132457.mp4'                                                                  

[09/05/25 12:44:50] INFO     Animation 1 : Partial movie file written in                   ]8;id=320881;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=131190;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/3267242197_1443                         
                             166991_2945085275.mp4'                                                                

[09/05/25 12:44:51] INFO     Animation 2 : Partial movie file written in                   ]8;id=756526;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=362775;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/3267242197_2802                         
                             432351_2375220172.mp4'                                                                

[09/05/25 12:44:52] INFO     Animation 3 : Partial movie file written in                   ]8;id=672956;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=280346;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/InvertingCircuits1/3267242197_2774                         
                             378717_1101682277.mp4'                                                                

...

                    INFO     Rendered InvertingCircuits1                                               ]8;id=376735;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=353278;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 7 animations                                                                   

[09/05/25 12:44:55] INFO     Generated 3 slides to                                                      ]8;id=280811;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=788007;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

[09/05/25 12:44:55] INFO     Generated 3 slides to                                                      ]8;id=711207;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=777905;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/InvertingCircuits1'                  

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=668007;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=488914;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

                    INFO     Slide 'InvertingCircuits1' configuration written in                        ]8;id=641143;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=107594;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/InvertingCircuits1.json'                   

## Our LB

In [ ]:
%%manim -ql OurLB

class OurLB(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"P_f", r"|\psi\rangle", r"S_f|\psi\rangle")

        ckt_flipped = OracleCircuit(r"P_f^{-1}", r"S_f|\psi\rangle", r"|\psi\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f^{-1}|\psi\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|\psi\rangle", r"S_f|\psi\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f^{-1}", r"|x\rangle|0\rangle", r"|x\rangle|f(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"P_f", r"|x\rangle|0\rangle", r"|x\rangle|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()

        FlipTransform(self, ckt, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $P_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

Manim Community v0.19.0

[09/05/25 12:44:57] INFO     Animation 0 : Partial movie file written in                   ]8;id=466399;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=190537;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/419893507_1812457392_2231324                         
                             57.mp4'                                                                               

[09/05/25 12:44:58] INFO     Animation 1 : Partial movie file written in                   ]8;id=937227;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=196756;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3267242197_1443166991_323893                         
                             5982.mp4'                                                                             

                    INFO     Animation 2 : Partial movie file written in                   ]8;id=13765;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=169920;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3267242197_1477149860_183347                         
                             9459.mp4'                                                                             

[09/05/25 12:44:59] INFO     Animation 3 : Partial movie file written in                   ]8;id=870428;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=199636;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/OurLB/3267242197_1443166991_115296                         
                             8606.mp4'                                                                             

...

                    INFO     Rendered OurLB                                                            ]8;id=504559;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=795316;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 17 animations                                                                  

[09/05/25 12:45:14] INFO     Generated 8 slides to                                                      ]8;id=459888;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=656219;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

[09/05/25 12:45:14] INFO     Generated 8 slides to                                                      ]8;id=387527;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=303739;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurLB'                               

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=118093;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=820100;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

                    INFO     Slide 'OurLB' configuration written in                                     ]8;id=495832;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=796653;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurLB.json'                                

# Search With In-Place Queries

## Our Alg

In [ ]:
%%manim -ql OurAlg

class OurAlg(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # title = Title("Search with In-Place Queries")
        # self.add(title)
        # self.next_slide()

        # bullets = Bullets(
        #     r"Can we mimic one iteration of Grover's algorithm?",
        #     1,
        #     r"In one iteration, we need to:",
        #     r".increase amplitude on $|x^*\rangle$ by $O\left(\frac{1}{\sqrt{N}}\right)$ \\ (Recall $x^* := f^{-1}(1)$ is our marked item)",
        #     r".use a constant number of in-place queries",
        #     font_scale=0.9, align_ref=title
        # )
        ## bullets[3][1][0][20:28].set_color(IN_PLACE)
        # bullets.play(self)

        steps = Steps()
        steps.add_step(r"\sum_{x \in [N]} |x\rangle", r"|0\rangle").shift(2*UP+2.5*RIGHT)
        steps.play(self, indices=[(-1,-1,0)])
        steps.play(self, indices=[(-1,-1,1)])
        steps.add_step(r"\sum_{x \in [N]} |x\rangle |x\rangle", cause=r"\text{CNOT}").play(self)
        steps.add_step(r"\sum_{x \in [N]}", r"|x\rangle |f(x)\rangle", cause=r"P_f").play(self)
        first_bad_step = len(steps)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |f(x)\rangle").play(self)
        steps.add_step(r"\sum_{x \in [N]} (-1)^{f(x) = 1} |x\rangle |0\rangle", cause=r"\text{???}").play(self)

        funcEras = MathTex(
            r"\xrightarrow[\text{Erasure}]{\text{Function}}"
        ).scale(steps.scale_factor).next_to(steps.prev_result(), LEFT).set_color(RED)
        self.play(ReplacementTransform(steps[-1][0], funcEras))
        self.next_slide()

        cross = Cross(steps[-1])
        self.play(Create(cross))
        self.next_slide()

        steps.add(cross)
        steps.play(self, indices=range(first_bad_step, len(steps)), animation=FadeOut)
        for _ in range(first_bad_step, len(steps)):
            steps.remove(steps[-1])
        self.next_slide()
        
        premark = MathTex(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle", 
            r" + |x^*\rangle |1\rangle"
        ).scale(steps.scale_factor).next_to(steps[-2][-1], DOWN).align_to(steps[-2][-1], LEFT)
        self.play(
            TransformMatchingShapes(steps.prev_result()[0], premark[0]),
            TransformMatchingShapes(steps.prev_result()[1], premark[1]),
            TransformFromCopy(steps.prev_result()[1], premark[2]),
        )
        steps[-1].remove(steps[-1][-1])
        steps[-1].add(premark)
        self.next_slide()

        # Mark
        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|x\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle"
        ).play(self)

        steps.add_step(
            r"\sum_{\substack{x \in [N] \\ f(x) \ne 1}}",
            r"|f(x)\rangle |f(x)\rangle \otimes |0\rangle", 
            r" + |x^*\rangle |1\rangle \otimes |1\rangle",
            cause = r"\text{ctrl-}P_f"
        )

Manim Community v0.19.0

[09/05/25 17:48:47] INFO     Animation 0 : Using cached data (hash :                           ]8;id=481378;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=558811;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             931832650_841098433_223132457)                                                        

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=748100;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=112606;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_2526439602)                                                     

[09/05/25 17:48:48] INFO     Animation 2 : Using cached data (hash :                           ]8;id=589970;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=942152;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_2902897767_1215932453)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=347701;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=801598;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             1905288516_1443166991_4253668531)                                                     

...

                    INFO     Rendered OurAlg                                                           ]8;id=702555;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=856572;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 20 animations                                                                  

[09/05/25 17:48:58] INFO     Generated 10 slides to                                                     ]8;id=55518;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=614368;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

[09/05/25 17:48:58] INFO     Generated 10 slides to                                                     ]8;id=198565;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=954622;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/OurAlg'                              

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=22543;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=517258;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

                    INFO     Slide 'OurAlg' configuration written in                                    ]8;id=651365;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=69703;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/OurAlg.json'                               

# Purgatory

## XORvsInPlace

In [ ]:
%%manim -ql XORvsInPlace

class XORvsInPlace(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        
        
        xor_bullets = [
            MathTex(r"\ket{x}\ket{a} \rightarrow \ket{x}\ket{f(x)}", tex_template=myTemplate),
            Text("Any f"),
            Text("Can erase f(x)"),
            Text("Self-inverse"),
            Text("Phase version")
        ]
        inplace_bullets = [
            MathTex(r"\ket{x} \rightarrow \ket{f(x)}", tex_template=myTemplate),
            Text("Bijections"),
            Text("Can erase x")
        ]
        
        xor_group = VGroup(*xor_bullets).arrange(DOWN)
        inplace_group = VGroup(*inplace_bullets).arrange(DOWN)

        tchart = MobjectTable(
            [[xor_group, inplace_group]],
            col_labels=[Text("XOR"), Text("In-Place")],
            arrange_in_grid_config={"row_alignments":'uu'})

        self.play(
            Write(tchart.get_horizontal_lines()), 
            Write(tchart.get_vertical_lines()),
            Write(tchart.get_col_labels()))
        self.next_slide()

        for i in range(len(xor_bullets)):
            self.play(Write(xor_bullets[i]))
            self.next_slide()

            if i < len(inplace_bullets):
                self.play(Write(inplace_bullets[i]))
                self.next_slide()

Manim Community v0.19.0

[09/05/25 12:47:16] INFO     Animation 0 : Partial movie file written in                   ]8;id=44214;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=173099;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/XORvsInPlace/419893507_140774604_2                         
                             23132457.mp4'                                                                         

[09/05/25 12:47:17] INFO     Animation 1 : Partial movie file written in                   ]8;id=826364;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=717717;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/XORvsInPlace/3267242197_1443166991                         
                             _3432205822.mp4'                                                                      

[09/05/25 12:47:18] INFO     Animation 2 : Partial movie file written in                   ]8;id=545397;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=956047;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/XORvsInPlace/3267242197_2047638319                         
                             _3606534566.mp4'                                                                      

[09/05/25 12:47:19] INFO     Animation 3 : Partial movie file written in                   ]8;id=289026;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=850266;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/XORvsInPlace/3267242197_1443166991                         
                             _174576785.mp4'                                                                       

...

                    INFO     Rendered XORvsInPlace                                                     ]8;id=836157;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=137732;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 18 animations                                                                  

[09/05/25 12:47:32] INFO     Generated 9 slides to                                                      ]8;id=648205;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=740865;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/XORvsInPlace'                        

[09/05/25 12:47:32] INFO     Generated 9 slides to                                                      ]8;id=308786;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=826018;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/XORvsInPlace'                        

                    INFO     Slide 'XORvsInPlace' configuration written in                              ]8;id=890604;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=259510;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/XORvsInPlace.json'                         

                    INFO     Slide 'XORvsInPlace' configuration written in                              ]8;id=4709;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=753242;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/XORvsInPlace.json'                         

## Kashefi

In [ ]:
%%manim -ql Kashefi

class Kashefi(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        ckt = OracleCircuit(r"S_f", r"|\psi\rangle", r"P_f|\psi\rangle")
        ckt_f = OracleCircuit(r"S_f", r"|x\rangle", r"|f(x)\rangle")

        ckt_flipped = OracleCircuit(r"S_f^{-1}", r"|f(x)\rangle", r"|x\rangle", flipped=True)
        flipped_ckts = [ckt_flipped]
        flipped_ckts.append(OracleCircuit(r"S_f^{-1}", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        flipped_ckts.append(OracleCircuit(r"S_f", r"|x\rangle", r"|f^{-1}(x)\rangle", flipped=True))
        
        self.play(FadeIn(ckt[0]), FadeIn(ckt[1]))
        self.next_slide()
        self.play(Succession([FadeIn(gate) for gate in ckt[-1]], run_time=1.2), FadeIn(ckt[2], run_time=1.2))
        self.next_slide()
        self.play(ReplacementTransform(ckt, ckt_f))
        self.next_slide()

        FlipTransform(self, ckt_f, ckt_flipped)
        self.next_slide()
        for i in range(len(flipped_ckts) - 1):
            self.play(ReplacementTransform(flipped_ckts[i], flipped_ckts[i+1]))
            self.next_slide()

        contradiction = Tex(r"Violates $\Omega(\sqrt{N})$ lower bound for computing $f^{-1}$ using $S_f$").next_to(flipped_ckts[-1][0], DOWN)
        self.play(Write(contradiction))
        self.next_slide()

Manim Community v0.19.0

[09/05/25 12:47:34] INFO     Animation 0 : Partial movie file written in                   ]8;id=101223;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=357101;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/419893507_1819904452_22313                         
                             2457.mp4'                                                                             

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=321162;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=222157;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3267242197_1443166991_9042                         
                             4229.mp4'                                                                             

[09/05/25 12:47:35] INFO     Animation 2 : Partial movie file written in                   ]8;id=660325;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=877023;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3267242197_3937844076_2150                         
                             33251.mp4'                                                                            

[09/05/25 12:47:36] INFO     Animation 3 : Partial movie file written in                   ]8;id=521665;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=850894;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/Kashefi/3267242197_1443166991_1024                         
                             975451.mp4'                                                                           

...

                    INFO     Rendered Kashefi                                                          ]8;id=509437;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=281259;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 15 animations                                                                  

[09/05/25 12:47:48] INFO     Generated 7 slides to                                                      ]8;id=336069;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=95867;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

[09/05/25 12:47:48] INFO     Generated 7 slides to                                                      ]8;id=671380;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=25367;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/Kashefi'                             

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=443170;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=749343;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

                    INFO     Slide 'Kashefi' configuration written in                                   ]8;id=538027;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=215998;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/Kashefi.json'                              

## AlgCkt

In [ ]:
%%manim -ql AlgCircuit

class AlgCircuit(Slide):
    def construct(self):
        self.wait_time_between_slides = 1

        # qcircuit
        qcircuit = r'''
        \Qcircuit @C=.5em @R=0.5em @!R {
         & & & & & & \mbox{\hspace{1.8em}\textit{Mark}} & & & & & & & & \mbox{\textit{Shift}} & & & & & & & & \mbox{\textit{Diffuse the Difference}} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \ctrl{4}  & \qw       & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \multigate{2}{\Ppi} & \ctrl{3}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \multigate{2}{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \ctrl{4}  & \qw       & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \ctrl{2}  & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \rstick{\ket{\psi_{t}}_\regA} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \ctrl{4}  & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ghost{\Ppi} & \qw       & \qw       & \ctrl{1}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \ghost{\diffusion} & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \push{\rule{0em}{1em}}
    \inputgroupv{2}{4}{0.5em}{2em}{\ket{\psi_{t-1}}_\regA \hspace{1.3em}} \\
    	& \lstick{\ket{0}_\regC \Big\{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \ctrlo{-1} & \ctrlo{1} & \ctrlo{2} & \ctrlo{3} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \gate{H}  & \ctrl{-1} & \gate{H}  & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \meter    & \cw       & \rstick{\text{\textit{Measure} }\regC} \\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \targ     & \qw       & \qw       & \multigate{2}{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \targ     & \qw       & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \qw       & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \rstick{\ket{0^n}_\regB} \push{\rule{0em}{1em}}\\
    	& \lstick{} & \qw       & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \targ     & \ghost{\Ppi} & \ctrlo{-1} & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \qw       & \qw       & \qw       & \targ     & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}} \qw & \push{\rule{0em}{1em}}
    \inputgroupv{6}{8}{0.5em}{2em}{\ket{0^n}_\regB \hspace{0.5em}}
    \gategroup{2}{29}{4}{29}{0em}{\}}
    \gategroup{6}{21}{8}{21}{0em}{\}}
    \gategroup{2}{4}{8}{10}{0.7em}{--}
    \gategroup{2}{13}{8}{18}{0.7em}{--}
    \gategroup{2}{21}{5}{25}{0.7em}{--}\\
    }
        '''
        label = MathTex(qcircuit, tex_template=myTemplate).scale(0.5)
        self.play(Write(label))
        self.wait()
        self.next_slide()

Manim Community v0.19.0

[09/05/25 12:48:09] INFO     Animation 0 : Partial movie file written in                   ]8;id=447436;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=567248;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/419893507_33113948_2231                         
                             32457.mp4'                                                                            

[09/05/25 12:48:13] INFO     Animation 1 : Partial movie file written in                   ]8;id=830189;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=642708;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/3267242197_722531385_31                         
                             08052316.mp4'                                                                         

[09/05/25 12:48:17] INFO     Animation 2 : Partial movie file written in                   ]8;id=836490;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=467471;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/AlgCircuit/3267242197_1443166991_2                         
                             134645725.mp4'                                                                        

                    INFO     Combining to Movie file.                                      ]8;id=995602;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=771536;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

...

                    INFO     Rendered AlgCircuit                                                       ]8;id=968506;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=43104;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 3 animations                                                                   

[09/05/25 12:48:18] INFO     Generated 1 slides to                                                      ]8;id=793452;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=774924;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

[09/05/25 12:48:18] INFO     Generated 1 slides to                                                      ]8;id=333391;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=938647;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/AlgCircuit'                          

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=352288;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=592071;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

                    INFO     Slide 'AlgCircuit' configuration written in                                ]8;id=619038;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=534625;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/AlgCircuit.json'                           

## StepThruOurAlgBar

In [ ]:
def get_HRY_amps(t, N=10):
    a_t = np.sin((t+1)*np.arctan(1/np.sqrt(N-1)))
    b_t = np.cos((t+1)*np.arctan(1/np.sqrt(N-1)))/np.sqrt(N-1)
    return a_t, b_t

In [ ]:
%%manim -ql StepThruOurAlgBar

class StepThruOurAlgBar(Slide):
    def construct(self):
        self.wait_time_between_slides = 1
        N = 10
        T = int(np.ceil(np.pi/2 * np.sqrt(N)))

        chart = BarChart(
            [1/np.sqrt(N)]*N,
            y_range=[-1, 1, 1],
            bar_colors=[IN_PLACE] + [GRAY]*(N-1),
            bar_names=[r"$x^*$"] + [""]*(N-1),
            x_axis_config={"font_size": 36},
        )
        y_axis_label = chart.get_y_axis_label(Tex("Amplitude").rotate(90*DEGREES), edge=LEFT, direction=LEFT, buff=0)

        self.play(DrawBorderThenFill(chart), DrawBorderThenFill(y_axis_label))
        self.next_slide()

        for t in range(1,T):
            a_t,b_t = get_HRY_amps(t,N)
            values = [a_t] + [b_t]*(N-1)
            self.play(chart.animate.change_bar_values(values))
            self.next_slide()


Manim Community v0.19.0

[09/05/25 12:48:20] INFO     Animation 0 : Partial movie file written in                   ]8;id=661289;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=901940;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/419893507_265164                         
                             8993_223132457.mp4'                                                                   

                    INFO     Animation 1 : Partial movie file written in                   ]8;id=367387;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=634410;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/3267242197_14431                         
                             66991_2544689240.mp4'                                                                 

[09/05/25 12:48:21] INFO     Animation 2 : Partial movie file written in                   ]8;id=676839;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=647407;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/3267242197_42156                         
                             14836_887163620.mp4'                                                                  

[09/05/25 12:48:22] INFO     Animation 3 : Partial movie file written in                   ]8;id=123336;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=860;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/ronakr/Documents/manimations/media/videos/manimations/                         
                             480p15/partial_movie_files/StepThruOurAlgBar/3267242197_14431                         
                             66991_3553262026.mp4'                                                                 

...

                    INFO     Rendered StepThruOurAlgBar                                                ]8;id=555099;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=276693;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 10 animations                                                                  

[09/05/25 12:48:28] INFO     Generated 5 slides to                                                      ]8;id=547235;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=250126;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

[09/05/25 12:48:28] INFO     Generated 5 slides to                                                      ]8;id=448108;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=329540;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#599\599]8;;\
                             '/home/ronakr/Documents/manimations/slides/files/StepThruOurAlgBar'                   

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=717615;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=224281;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    

                    INFO     Slide 'StepThruOurAlgBar' configuration written in                         ]8;id=251116;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py\base.py]8;;\:]8;id=991059;file:///home/ronakr/Documents/manimations/.venv/lib/python3.13/site-packages/manim_slides/slide/base.py#611\611]8;;\
                             '/home/ronakr/Documents/manimations/slides/StepThruOurAlgBar.json'                    